In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from scipy.stats import expon, reciprocal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import deap
import skopt
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import save_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l1, l2, l1_l2
from sklearn.model_selection import ParameterGrid

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import joblib

def preprocess_data(file_path, is_train=True):
    df = pd.read_csv(file_path)
    
    # Convert binary categorical features to 0 and 1
    binary_features = ['CryoSleep', 'VIP']
    df[binary_features] = df[binary_features].astype(bool).astype(int)
    
    # Feature Engineering
    df['TotalSpending'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    
    # Conditionally set spending-related features to 0 for passengers in cryosleep
    spending_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df.loc[df['CryoSleep'] == 1, spending_features] = 0
    
    # Create interaction features
    df['HomePlanet_TotalSpending'] = df['HomePlanet'].astype(str) + '_' + df['TotalSpending'].astype(str)
    df['Destination_TotalSpending'] = df['Destination'].astype(str) + '_' + df['TotalSpending'].astype(str)
    
    # Extract components from 'Cabin'
    if 'Cabin' in df.columns:
        df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = df['Cabin'].str.split('/', expand=True)
        df['Cabin_Side'] = df['Cabin_Side'].map({'P': 1, 'S': 0})
        df['Cabin_Number'] = pd.to_numeric(df['Cabin_Number'], errors='coerce')
        df.drop('Cabin', axis=1, inplace=True)
    
    # One-hot encode multi-category features
    multi_cat_features = ['HomePlanet', 'Destination']
    if is_train:
        one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
        encoded_features = one_hot_encoder.fit_transform(df[multi_cat_features])
        encoded_feature_names = one_hot_encoder.get_feature_names_out(multi_cat_features)
        joblib.dump(one_hot_encoder, 'one_hot_encoder.pkl')
    else:
        one_hot_encoder = joblib.load('one_hot_encoder.pkl')
        encoded_features = one_hot_encoder.transform(df[multi_cat_features])
        encoded_feature_names = one_hot_encoder.get_feature_names_out(multi_cat_features)
    
    encoded_features_df = pd.DataFrame(encoded_features.toarray(), columns=encoded_feature_names)
    df = pd.concat([df, encoded_features_df], axis=1)
    df.drop(multi_cat_features, axis=1, inplace=True)
    
    # Imputation and Scaling
    numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin_Number', 'Cabin_Side', 'TotalSpending']
    if is_train:
        imputer = KNNImputer(n_neighbors=5)
        scaler = StandardScaler()
        df[numeric_features] = imputer.fit_transform(df[numeric_features])
        df[numeric_features] = scaler.fit_transform(df[numeric_features])
        joblib.dump(imputer, 'imputer.pkl')
        joblib.dump(scaler, 'scaler.pkl')
    else:
        imputer = joblib.load('imputer.pkl')
        scaler = joblib.load('scaler.pkl')
        df[numeric_features] = imputer.transform(df[numeric_features])
        df[numeric_features] = scaler.transform(df[numeric_features])
    
    if is_train:
        # Convert 'Transported' to integer (True=1, False=0) for modeling
        df['Transported'] = df['Transported'].astype(int)
        
        # Save the list of features used for training
        train_features = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Transported', 'Cabin_Deck', 'HomePlanet_TotalSpending', 'Destination_TotalSpending']]
        joblib.dump(train_features, 'train_features.pkl')
    
    return df

In [37]:
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np

train_df = preprocess_data('csv_files/train.csv', is_train=True)
test_df = preprocess_data('csv_files/test.csv', is_train=False)

def create_model(input_shape, layers, units, activation, dropout_rate, l1_reg, l2_reg, learning_rate):
    model = Sequential()
    for i in range(layers):
        if i == 0:
            model.add(Dense(units, activation=activation, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), input_shape=(input_shape,)))
        else:
            model.add(Dense(units, activation=activation, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate(model, X_train, y_train, X_val, y_val, batch_size, epochs, class_weight, lr_schedule):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    lr_scheduler = LearningRateScheduler(lr_schedule)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=(X_val, y_val), class_weight=class_weight, 
              callbacks=[early_stopping, lr_scheduler])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    return val_loss, val_acc

# Prepare features and target for the model
features = [col for col in train_df.columns if col not in ['PassengerId', 'Name', 'Transported', 'Cabin_Deck', 'HomePlanet_TotalSpending', 'Destination_TotalSpending']]
X = train_df[features]
y = train_df['Transported']

# Handling class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Define the focused hyperparameter search space
dim_layers = Integer(low=2, high=6, name='layers')
dim_units = Integer(low=256, high=512, name='units')
dim_activation = Categorical(categories=['tanh', 'relu', 'selu', 'swish'], name='activation')
dim_dropout = Real(low=0.1, high=0.3, prior='uniform', name='dropout_rate')
dim_l1_reg = Real(low=1e-6, high=1e-4, prior='log-uniform', name='l1_reg')
dim_l2_reg = Real(low=1e-6, high=1e-4, prior='log-uniform', name='l2_reg')
dim_learning_rate = Real(low=1e-3, high=1e-2, prior='log-uniform', name='learning_rate')

dimensions = [dim_layers, dim_units, dim_activation, dim_dropout, dim_l1_reg, dim_l2_reg, dim_learning_rate]

def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.exp(-0.1)

@use_named_args(dimensions=dimensions)
def fitness(layers, units, activation, dropout_rate, l1_reg, l2_reg, learning_rate):
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    val_accuracies = []
    
    for train_index, val_index in kfold.split(X_resampled, y_resampled):
        X_train, X_val = X_resampled.iloc[train_index], X_resampled.iloc[val_index]
        y_train, y_val = y_resampled.iloc[train_index], y_resampled.iloc[val_index]
        
        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = dict(enumerate(class_weights))
        
        model = create_model(input_shape=X_train.shape[1], layers=layers, units=units, activation=activation, 
                             dropout_rate=dropout_rate, l1_reg=l1_reg, l2_reg=l2_reg, learning_rate=learning_rate)
        _, val_acc = train_and_evaluate(model, X_train, y_train, X_val, y_val, 
                                        batch_size=32, epochs=100, class_weight=class_weight_dict, lr_schedule=lr_schedule)
        val_accuracies.append(val_acc)
    
    return -np.mean(val_accuracies)

# Perform focused architecture search using Bayesian Optimization
n_calls = 10
best_result = gp_minimize(func=fitness, dimensions=dimensions, n_calls=n_calls, 
                          random_state=42, verbose=True)

# Retrieve the best hyperparameters
best_params = {
    'layers': best_result.x[0],
    'units': best_result.x[1],
    'activation': best_result.x[2],
    'dropout_rate': best_result.x[3],
    'l1_reg': best_result.x[4],
    'l2_reg': best_result.x[5],
    'learning_rate': best_result.x[6]
}
best_val_acc = -best_result.fun

# Train the final model with the best hyperparameters
model_enhanced = create_model(input_shape=X_resampled.shape[1], **best_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_resampled), y=y_resampled)
class_weight_dict = dict(enumerate(class_weights))
model_enhanced.fit(X_resampled, y_resampled, batch_size=32, epochs=100, class_weight=class_weight_dict, callbacks=[LearningRateScheduler(lr_schedule)])

print(f"Best Validation Accuracy: {best_val_acc}")
print(f"Best Hyperparameters: {best_params}")

Iteration No: 1 started. Evaluating function at random point.


/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7381 - loss: 0.6496 - val_accuracy: 0.7928 - val_loss: 0.5535 - learning_rate: 0.0029
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7722 - loss: 0.5664 - val_accuracy: 0.7979 - val_loss: 0.4974 - learning_rate: 0.0029
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7916 - loss: 0.5119 - val_accuracy: 0.7997 - val_loss: 0.4679 - learning_rate: 0.0029
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7854 - loss: 0.4993 - val_accuracy: 0.8059 - val_loss: 0.4512 - learning_rate: 0.0029
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7911 - loss: 0.4734 - val_accuracy: 0.8037 - val_loss: 0.4464 - learning_rate: 0.0029
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8000 - loss: 0.4585 - val_accuracy: 0.7922 - val_loss: 0.4567 - learning_rate: 0.0029
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7959 - loss: 0.

/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7535 - loss: 0.6318 - val_accuracy: 0.7756 - val_loss: 0.5475 - learning_rate: 0.0029
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7772 - loss: 0.5418 - val_accuracy: 0.7801 - val_loss: 0.5016 - learning_rate: 0.0029
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8035 - loss: 0.4967 - val_accuracy: 0.7830 - val_loss: 0.5002 - learning_rate: 0.0029
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7877 - loss: 0.4862 - val_accuracy: 0.7767 - val_loss: 0.4833 - learning_rate: 0.0029
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7943 - loss: 0.4690 - val_accuracy: 0.7898 - val_loss: 0.4619 - learning_rate: 0.0029
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7840 - loss: 0.4702 - val_accuracy: 0.7830 - val_loss: 0.4698 - learning_rate: 0.0029
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7987 - loss: 0.4490 - val_a

...

219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7389 - loss: 0.5896 - val_accuracy: 0.7796 - val_loss: 0.5095 - learning_rate: 0.0027
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7927 - loss: 0.5173 - val_accuracy: 0.7836 - val_loss: 0.4904 - learning_rate: 0.0027
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7866 - loss: 0.4880 - val_accuracy: 0.7836 - val_loss: 0.4690 - learning_rate: 0.0027
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7951 - loss: 0.4573 - val_accuracy: 0.8018 - val_loss: 0.4729 - learning_rate: 0.0027
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8061 - loss: 0.4431 - val_accuracy: 0.8058 - val_loss: 0.4586 - learning_rate: 0.0027
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8057 - loss: 0.4428 - val_accuracy: 0.8081 - val_loss: 0.4438 - learning_rate: 0.0027
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7979 - loss: 0.4397 - val_a

/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7420 - loss: 0.5847 - val_accuracy: 0.7961 - val_loss: 0.4752 - learning_rate: 0.0027
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7846 - loss: 0.5037 - val_accuracy: 0.7893 - val_loss: 0.4734 - learning_rate: 0.0027
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7985 - loss: 0.4881 - val_accuracy: 0.7790 - val_loss: 0.4702 - learning_rate: 0.0027
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7902 - loss: 0.4652 - val_accuracy: 0.7955 - val_loss: 0.4480 - learning_rate: 0.0027
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8039 - loss: 0.4491 - val_accuracy: 0.7990 - val_loss: 0.4363 - learning_rate: 0.0027
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7952 - loss: 0.4404 - val_accuracy: 0.7990 - val_loss: 0.4466 - learning_rate: 0.0027
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8074 - loss: 0.4191 - val_a

/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7521 - loss: 0.6452 - learning_rate: 0.0029
Epoch 2/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7851 - loss: 0.5378 - learning_rate: 0.0029
Epoch 3/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7827 - loss: 0.5083 - learning_rate: 0.0029
Epoch 4/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7802 - loss: 0.4806 - learning_rate: 0.0029
Epoch 5/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7859 - loss: 0.4705 - learning_rate: 0.0029
Epoch 6/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7880 - loss: 0.4619 - learning_rate: 0.0029
Epoch 7/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7977 - loss: 0.4544 - learning_rate: 0.0029
Epoch 8/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7911 - loss: 0.4550 - learning_rate: 0.0029
Epoch 9/100
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8072 - loss: 0.4396 - learning_rate: 0.0029
Epoch 10/100



Run 1:


Not so advanced 
Best Validation Accuracy: 0.7998849749565125
Best Hyperparameters: {'layers': [379], 'activation': 'tanh', 'dropout_rate': 0.10031150633640573, 'learning_rate': 0.009647685075720105}


Run 2:


**Much more advanced**
Best Validation Accuracy: 0.8081
Best Hyperparameters: {'layers': [312, 312], 'activation': 'tanh', 'dropout_rate': 0.1, 'l1_reg': 5.778749032107547e-05, 'l2_reg': 1e-05, 'learning_rate': 0.01} what does this tell us? How can we incorperate these findings to enhancing performance?

Run 3:

**Honing In even More**
Best Validation Accuracy: 0.8092737674713135


Best Hyperparameters: {'layers': 5, 'units': 303, 'activation': 'swish', 'dropout_rate': 0.2193700315892974, 'l1_reg': 7.792297153882995e-06, 'l2_reg': 1.5847101210439079e-06, 'learning_rate': 0.002879047909793294}

In [40]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import joblib
print(f"Best Validation Accuracy: {best_val_acc}")
print(f"Best Hyperparameters: {best_params}")

# Preprocess the test data using the same steps as the training data
test_df = preprocess_data('csv_files/test.csv', is_train=False)

# Load the features used during training
train_features = joblib.load('train_features.pkl')

# Select the features used by the model, ensuring they are the same as those used in training
features = [col for col in train_features if col in test_df.columns]

# Ensure that X_test is a DataFrame with the correct numerical types
X_test = test_df[features].copy()

# Convert X_test to a NumPy array, which is the expected format for TensorFlow models
X_test_np = X_test.values

# Predict using the enhanced model
y_pred_test_proba = model_enhanced.predict(X_test_np)
y_pred_test = (y_pred_test_proba > 0.5).astype(int).flatten()

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_pred_test
})

# Convert predictions back to boolean (True/False) if necessary
submission_df['Transported'] = submission_df['Transported'].astype(bool)

# Save the submission file
submission_df.to_csv('neuralnetbayesadvancedfocused.csv', index=False)

Best Validation Accuracy: 0.8092737674713135
Best Hyperparameters: {'layers': 5, 'units': 303, 'activation': 'swish', 'dropout_rate': 0.2193700315892974, 'l1_reg': 7.792297153882995e-06, 'l2_reg': 1.5847101210439079e-06, 'learning_rate': 0.002879047909793294}
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
